In [2]:
import praw
import pandas as pd

In [3]:
# get reddit secret & script i've saved as .txt in the working directory

reddit_script = open('reddit_oa_script.txt')

reddit_secret = open('reddit_oa_secret.txt')

In [5]:
# user_agent is the name of the app

reddit  = praw.Reddit(client_id='reddit_script',
                          client_secret='reddit_secret',
                          user_agent='get_corpora',
                          username='neurodivergent_ai',
                          password='reddit_id.txt')

In [6]:
# get the subreddit

subreddit_1 = reddit.subreddit('The_Donald')

# get the top threads

subreddit_1_top = subreddit_1.top(limit=1000)